In [1]:
from competition_utils import *
from models.base_config import BaseConfig

# 제출 함수를 리스트로 정의 (submission_N_학번)
# 깃허브에는 최대 3개 업로드 데모시 하이퍼파라미터 튜닝 후 1개만 제출
# submission_functions = ['submission_YOLOv8n', 'submission_YOLOv9t', 'submission_YOLOv10n', 'submission_YOLO11n', 'submission_YOLO12n']  
submission_functions = ['submission_YOGAn']
for submission_function in submission_functions:
    exec(f"from submissions.{submission_function} import {submission_function}")

# 분석 방향에 따라 iteration 수 수정
# 평가시에는 데모와 다른 스플릿 시드를 만들어 [1, 10]로 진행 예정
iterations = [1, 1] 

# 'Crown Detection'는 예제 데이터셋, 데모 및 평가시에는 'CV_Competition'
Dataset_Name = 'airplane' 

# 결과를 모아 하나의 csv 파일에 저장
results_df = pd.DataFrame(columns=[
    'Experiment Time', 'Iteration', 'Submission Function', 
    'IoU', 'Dice', 'Precision', 'Recall', 'Output Json Path',
])
csv_filename = os.path.join(BaseConfig.get_output_dir(), f"Evaluation_Results_{BaseConfig.get_experiment_time()}.csv")
json_foldername = "json"
vis_foldername = "vis"

os.makedirs(os.path.join(BaseConfig.get_output_dir(), json_foldername), exist_ok=True)
os.makedirs(os.path.join(BaseConfig.get_output_dir(), vis_foldername), exist_ok=True)

# 서로 다른 iteration, submission function으로 실험 진행
for iteration in range(iterations[0], iterations[1]+1):
    
    yaml_path = f'Datasets/{Dataset_Name}/data_iter_{iteration:02d}.yaml'
    
    for submission_function in submission_functions:
        
        ex_time = datetime.now().strftime('%y%m%d_%H%M%S')
        output_json_path = os.path.join(BaseConfig.get_output_dir(), json_foldername, f"{ex_time}_{submission_function}_Iter_{iteration}_detection_results.json")
        
        globals()[submission_function](yaml_path, output_json_path)
        
        labels_dir = f"Datasets/{Dataset_Name}/labels"  
        vis_output_dir = os.path.join(BaseConfig.get_output_dir(), vis_foldername, f"{ex_time}_visualization_results")
        image_level_result_path =os.path.join(BaseConfig.get_output_dir(), json_foldername, f"{ex_time}_{submission_function}_Iter_{iteration}_image_level_results.json")
        stats = eval_and_vis(yaml_path, output_json_path, labels_dir, image_level_result_path, vis_output_dir, vis=True) # 분석 방향에 따라 vis=True 설정
        new_row = {
            'Experiment Time': ex_time,
            'Iteration': iteration,
            'Submission Function': submission_function,
            'IoU': stats['IoU']['avg'],
            'Dice': stats['Dice']['avg'],
            'Precision': stats['Precision']['avg'],
            'Recall': stats['Recall']['avg'],
            'Output Json Path': output_json_path,
        }
        results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)
        results_df.to_csv(csv_filename, index=False)

/home/gamejoongsa/anaconda3/envs/CV/lib/python3.10/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


train: weights=, cfg=models/YOGAn/pkgs/yoga_models/YOGA-n.yaml, data=Datasets/airplane/data_iter_01.yaml, hyp=output/250420_011426/hyp_yogan.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=cuda:0, multi_scale=False, single_cls=True, adam=False, sync_bn=False, workers=8, project=output/250420_011426/train, name=250420_011427_yogan_airplane_Iter_1, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (offline), for updates see https://github.com/ultralytics/yolov5


YOLOv5 🚀 6e61f54 torch 2.8.0.dev20250417+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16302.5625MB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.9, weight_decay=0.0001, warmup_epochs=2.0, warmup_momentum=0.85, warmup_bias_lr=0.1, box=5.0, cls=0.3, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=10.0, translate=0.1, scale=0.5, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.5, mosaic=1.0, mixup=0.0, copy_paste=0.0
TensorBoard: Start with 'tensorboard --logdir output/250420_011426/train', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      2640  yoga_models.common.Conv                 [3, 24, 6, 2, 2]              
  1                -1  1      5880  yoga_models.common.GhostConv            [24, 48, 3, 2]                
  2                -1  2      6420  yoga_models.

Plotting labels... 


Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to output/250420_011426/train/250420_011427_yogan_airplane_Iter_1
Starting training for 20 epochs...

     Epoch   gpu_mem       box       obj       cls    labels  img_size


...

Fusing layers... 
Model Summary: 618 layers, 1820905 parameters, 0 gradients, 4.3 GFLOPs
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100%|██████████| 11/11 [00:02<00:00,  4.81it/s]


                 all        351        379      0.273     0.0369     0.0339    0.00831


Results saved to output/250420_011426/train/250420_011427_yogan_airplane_Iter_1
YOLOv5 🚀 6e61f54 torch 2.8.0.dev20250417+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16302.5625MB)

Fusing layers... 
Model Summary: 618 layers, 1820905 parameters, 0 gradients, 4.3 GFLOPs


결과가 output/250420_011426/json/250420_011426_submission_YOGAn_Iter_1_detection_results.json에 저장되었습니다.


/tmp/ipykernel_134752/939340058.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)


In [ ]:
# 분석 방향에 따라 전체 결과를 모아 평균-표준편차-통계테스트 결과 저장 

keep_columns = ['Iteration','Submission Function']
keep_measures = ['IoU','Dice','Precision','Recall']  

reduction = 'Iteration'
row = 'Measure Type'
column = 'Submission Function'
reference_column = 'submission_YOLOv8n'

custom_fmt_template = '{mean_fmt} ± {std_fmt} {significance}'
significance_levels = [0.1, 0.05, 0.01]
decimal_places = 3
transpose = True
make_tables(csv_filename, keep_columns, keep_measures, reduction, row, column,  
            reference_column, custom_fmt_template, significance_levels, decimal_places, transpose)